In [1]:
import pandas as pd
import numpy as np
from glob import glob
from path import Path
import json
import seaborn as sns
import prody
from multiprocessing import Pool
import tqdm
import itertools
import json
from rdkit import Chem
from rdkit.Chem import AllChem
from io import StringIO
from collections import OrderedDict, Counter, defaultdict
import traceback
import urllib
from multiprocessing import Pool
import seaborn as sns
from copy import deepcopy
import pybel
import contextlib
import subprocess

from gemmi import cif

from pdbtools import utils

RDKit WARNING: [17:19:40] Enabling RDKit 2019.09.3 jupyter extensions
[17:19:40] Enabling RDKit 2019.09.3 jupyter extensions
/home/ignatovmg/projects/pdb_tools/venv/lib/python3.7/site-packages/Bio/SubsMat/__init__.py:131: BiopythonDeprecationWarning: Bio.SubsMat has been deprecated, and we intend to remove it in a future release of Biopython. As an alternative, please consider using Bio.Align.substitution_matrices as a replacement, and contact the Biopython developers if you still need the Bio.SubsMat module.
  BiopythonDeprecationWarning,


In [2]:
dataset_dir = Path('data')
msa_dir = dataset_dir / '15k/folding/MMSEQ_submission_second_try'
cif_dir = dataset_dir / '15k/folding/cifs'

In [3]:
def format_request(entities):
    request = '''
    {
      polymer_entities(entity_ids: [?]){
        rcsb_cluster_membership {
          cluster_id,
          identity
        }
        rcsb_polymer_entity_container_identifiers {
            rcsb_id
        }
        entry {
          exptl{
            method
          }
          rcsb_accession_info {
            initial_release_date,
            deposit_date
          }
          rcsb_entry_info {
            resolution_combined
          }
        }
      }
    }
    '''
    
    string = '"' + '", "'.join(entities) + '"'
    request = string.join(request.split('?'))
    url = 'https://data.rcsb.org/graphql?query=' + urllib.parse.quote_plus(request)
    return url


def get_rscb_json(url, num_tries=5, timeout=5):
    for i in range(num_tries):
        try:
            with urllib.request.urlopen(url) as f:
                result = json.loads(f.read().decode())
        except urllib.error.HTTPError:
            print('HTTPError. Trying again' if i != num_tries - 1 else '. Reached max number of trials')
            result = {}
            sleep(timeout)
        if result:
            break
    return result


def load_entities_list_in_chunks(entities_list, chunk_size=1000, timeout=5):
    out = []
    for idx in tqdm.tqdm(range(0, len(entities_list), chunk_size)):
        chunk = entities_list[idx:idx + chunk_size]
        url = format_request(chunk)
        chunk_result = get_rscb_json(url)
        out += chunk_result['data']['polymer_entities']
    return out


#url = format_request(["4HHB_1", "1H8E_4"])
#get_rscb_json(url)
'3EHD_1'
load_entities_list_in_chunks(["4HHB_1", "1H8E_4"])

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 17.65it/s]


[{'rcsb_cluster_membership': [{'cluster_id': 98, 'identity': 100},
   {'cluster_id': 115, 'identity': 95},
   {'cluster_id': 122, 'identity': 90},
   {'cluster_id': 33, 'identity': 70},
   {'cluster_id': 8, 'identity': 50},
   {'cluster_id': 14, 'identity': 30}],
  'rcsb_polymer_entity_container_identifiers': {'rcsb_id': '4HHB_1'},
  'entry': {'exptl': [{'method': 'X-RAY DIFFRACTION'}],
   'rcsb_accession_info': {'initial_release_date': '1984-07-17T00:00:00Z',
    'deposit_date': '1984-03-07T00:00:00Z'},
   'rcsb_entry_info': {'resolution_combined': [1.74]}}},
 {'rcsb_cluster_membership': [{'cluster_id': 954, 'identity': 100},
   {'cluster_id': 1052, 'identity': 95},
   {'cluster_id': 1068, 'identity': 90},
   {'cluster_id': 1118, 'identity': 70},
   {'cluster_id': 1144, 'identity': 50},
   {'cluster_id': 860, 'identity': 30}],
  'rcsb_polymer_entity_container_identifiers': {'rcsb_id': '1H8E_4'},
  'entry': {'exptl': [{'method': 'X-RAY DIFFRACTION'}],
   'rcsb_accession_info': {'initia

In [4]:
#load_entities_list_in_chunks(["3EHD_1", '4HHB_1'])

In [5]:
case_msa_dirs = sorted(msa_dir.glob('*'))

In [6]:
entities_list = [x.basename().split('.')[0].upper() for x in case_msa_dirs]
entities_rcsb_info = load_entities_list_in_chunks(entities_list)
entities_rcsb_info = {x['rcsb_polymer_entity_container_identifiers']['rcsb_id']: x for x in entities_rcsb_info}

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 15/15 [00:33<00:00,  2.24s/it]


In [7]:
list(entities_rcsb_info.values())[0]

{'rcsb_cluster_membership': [{'cluster_id': 58960, 'identity': 100},
  {'cluster_id': 66123, 'identity': 95},
  {'cluster_id': 16409, 'identity': 90},
  {'cluster_id': 11075, 'identity': 70},
  {'cluster_id': 11542, 'identity': 50},
  {'cluster_id': 10275, 'identity': 30}],
 'rcsb_polymer_entity_container_identifiers': {'rcsb_id': '19HC_1'},
 'entry': {'exptl': [{'method': 'X-RAY DIFFRACTION'}],
  'rcsb_accession_info': {'initial_release_date': '1999-12-01T00:00:00Z',
   'deposit_date': '1998-12-01T00:00:00Z'},
  'rcsb_entry_info': {'resolution_combined': [1.8]}}}

In [ ]:
cases = []

letters = list('ACDEFGHIKLMNPQRSTVWYX')
assert len(letters) == 21, len(letters)


def loop_to_list(block, category):
    cat = block.find_mmcif_category(category)
    assert len(cat) > 0, (category, 'does not exist')
    out = []
    for row in cat:
        row_dict = OrderedDict()
        for key in cat.tags:
            row_dict[key] = row[key[cat.prefix_length:]]
        out.append(row_dict)
    return out


for case_msa_dir in tqdm.tqdm(case_msa_dirs):
    #case_msa_dir = case_msa_dirs[0]
    pdb_id = case_msa_dir.basename()[:4]
    case_entity_id = case_msa_dir.basename().split('.')[0][5:]
    
    cif_file = cif_dir / pdb_id + '.cif'
    try:
        ag = prody.parseMMCIF(cif_file)
        assert ag is not None
    except KeyboardInterrupt:
        raise
    except:
        print('Could not parse', cif_file)
        traceback.print_exc()
        continue
    
    cif_parsed = cif.read_file(cif_file).sole_block()
    entity_id_to_asym_id = defaultdict(list)
    table = cif_parsed.find_mmcif_category('_struct_asym')
    for asym_id, entity_id in zip(table.find_column('id'), table.find_column('entity_id')):
        entity_id_to_asym_id[entity_id].append(asym_id)
        
    # parse entity poly seq, to compare with the pdbx_seq_one_letter_code_can
    # this is necessary because for example in 1t6j_1 
    # len(_entity_poly.pdbx_seq_one_letter_code) == len(_entity_poly_seq) == 174 and
    # len(_entity_poly.pdbx_seq_one_letter_code_can) == 176 due to some mistake,
    # so we skip such cases
    entity_poly_seq = loop_to_list(cif_parsed, '_entity_poly_seq')
    entity_poly_seq = [x for x in entity_poly_seq if x['_entity_poly_seq.entity_id'] == case_entity_id]

    # alt locations have the same residue number,
    # keep only the first one to match the pdbx_seq_one_letter_code_can string
    seen_ids = []
    buf = []
    for item in entity_poly_seq:
        if item['_entity_poly_seq.num'] not in seen_ids:
            seen_ids.append(item['_entity_poly_seq.num'])
            buf.append(item)
    entity_poly_seq = buf

    entity_info = None
    table = cif_parsed.find_mmcif_category('_entity_poly')
    for entity_id, pdbx_strand_id, pdbx_seq_one_letter_code_can in zip(
        table.find_column('entity_id'), 
        table.find_column('pdbx_strand_id'), 
        table.find_column('pdbx_seq_one_letter_code_can')
    ):
        #print(pdbx_seq_one_letter_code_can)
        seq_conv = pdbx_seq_one_letter_code_can.replace('\n', '').replace(';', '').replace('U', 'X').replace('O', 'X')
        if case_entity_id == entity_id:
            entity_info = OrderedDict(
                entity_id=entity_id,
                pdbx_strand_ids=pdbx_strand_id.split(','),
                asym_ids=entity_id_to_asym_id[entity_id],
                pdbx_seq_one_letter_code_can=seq_conv
            )
            assert all([aa in letters for aa in seq_conv]), (case_msa_dir, seq_conv, set(seq_conv) - set(letters))
    assert entity_info is not None, (cif_file, case_entity_id)
    
    if len(entity_info['pdbx_seq_one_letter_code_can']) != len(entity_poly_seq):
        print('Mismatching sequences in', cif_file, case_entity_id)
        continue
        
    # TODO: res < 9A, add seq clusters 40%
    rcsb_key = case_msa_dir.basename().split('.')[0].upper()
    if rcsb_key not in entities_rcsb_info:
        print('No info for ', rcsb_key)
        continue
    
    case_dict = OrderedDict(
        pdb_id=pdb_id,
        entity_id=case_entity_id,
        entity_info=entity_info,
        cif_file=cif_file.relpath(dataset_dir),
        a3m_files=[
            case_msa_dir.relpath(dataset_dir) / 'uniref.a3m',
            case_msa_dir.relpath(dataset_dir) / 'bfd.mgnify30.metaeuk30.smag30.a3m'
        ],
        rcsb_info=entities_rcsb_info[rcsb_key]
    )
    cases.append(case_dict)

  2%|██▉                                                                                                                                                                | 265/14950 [00:21<19:48, 12.35it/s]Traceback (most recent call last):
  File "/tmp/ipykernel_79549/2442026332.py", line 26, in <module>
    ag = prody.parseMMCIF(cif_file)
  File "/home/ignatovmg/projects/pdb_tools/venv/lib/python3.7/site-packages/prody/proteins/ciffile.py", line 110, in parseMMCIF
    result = parseMMCIFStream(cif, chain=chain, **kwargs)
  File "/home/ignatovmg/projects/pdb_tools/venv/lib/python3.7/site-packages/prody/proteins/ciffile.py", line 181, in parseMMCIFStream
    altloc, header)
  File "/home/ignatovmg/projects/pdb_tools/venv/lib/python3.7/site-packages/prody/proteins/ciffile.py", line 233, in _parseMMCIFLines
    models.append(line.split()[fields['pdbx_PDB_model_num']])
KeyError: 'pdbx_PDB_model_num'
  2%|██▉                                                                                    

Could not parse data/15k/folding/cifs/1dhk.cif


  2%|███▊                                                                                                                                                               | 352/14950 [00:27<22:36, 10.76it/s]Traceback (most recent call last):
  File "/tmp/ipykernel_79549/2442026332.py", line 26, in <module>
    ag = prody.parseMMCIF(cif_file)
  File "/home/ignatovmg/projects/pdb_tools/venv/lib/python3.7/site-packages/prody/proteins/ciffile.py", line 110, in parseMMCIF
    result = parseMMCIFStream(cif, chain=chain, **kwargs)
  File "/home/ignatovmg/projects/pdb_tools/venv/lib/python3.7/site-packages/prody/proteins/ciffile.py", line 181, in parseMMCIFStream
    altloc, header)
  File "/home/ignatovmg/projects/pdb_tools/venv/lib/python3.7/site-packages/prody/proteins/ciffile.py", line 233, in _parseMMCIFLines
    models.append(line.split()[fields['pdbx_PDB_model_num']])
KeyError: 'pdbx_PDB_model_num'
  2%|███▉                                                                                   

Could not parse data/15k/folding/cifs/1el4.cif


  9%|█████████████▉                                                                                                                                                    | 1286/14950 [01:55<18:43, 12.17it/s]Traceback (most recent call last):
  File "/tmp/ipykernel_79549/2442026332.py", line 26, in <module>
    ag = prody.parseMMCIF(cif_file)
  File "/home/ignatovmg/projects/pdb_tools/venv/lib/python3.7/site-packages/prody/proteins/ciffile.py", line 110, in parseMMCIF
    result = parseMMCIFStream(cif, chain=chain, **kwargs)
  File "/home/ignatovmg/projects/pdb_tools/venv/lib/python3.7/site-packages/prody/proteins/ciffile.py", line 181, in parseMMCIFStream
    altloc, header)
  File "/home/ignatovmg/projects/pdb_tools/venv/lib/python3.7/site-packages/prody/proteins/ciffile.py", line 233, in _parseMMCIFLines
    models.append(line.split()[fields['pdbx_PDB_model_num']])
KeyError: 'pdbx_PDB_model_num'


...

 10%|████████████████▉                                                                                                                                                 | 1563/14950 [02:18<17:59, 12.40it/s]

Mismatching sequences in data/15k/folding/cifs/1rrx.cif 1


 11%|██████████████████▎                                                                                                                                               | 1694/14950 [02:29<19:39, 11.24it/s]

Mismatching sequences in data/15k/folding/cifs/1t6j.cif 1


 13%|█████████████████████▌                                                                                                                                            | 1988/14950 [02:54<15:04, 14.34it/s]

In [32]:
utils.write_json(cases, 'data/15k/folding/debug_15k.json')

In [46]:
len(cases)

7417

In [33]:
cases[100]

OrderedDict([('pdb_id', '1ci8'),
             ('entity_id', '1'),
             ('entity_info',
              OrderedDict([('entity_id', '1'),
                           ('pdbx_strand_ids', ['A', 'B']),
                           ('asym_ids', ['A', 'B']),
                           ('pdbx_seq_one_letter_code_can',
                            'MTAASLDPTAFSLDAASLAARLDAVFDQALRERRLVGAVAIVARHGEILYRRAQGLADREAGRPMREDTLFRLASVTKPIVALAVLRLVARGELALDAPVTRWLPEFRPRLADGSEPLVTIHHLLTHTSGLGYWLLEGAGSVYDRLGISDGIDLRDFDLDENLRRLASAPLSFAPGSGWQYSLALDVLGAVVERATGQPLAAAVDALVAQPLGMRDCGFVSAEPERFAVPYHDGQPEPVRMRDGIEVPLPEGHGAAVRFAPSRVFEPGAYPSGGAGMYGSADDVLRALEAIRANPGFLPETLADAARRDQAGVGAETRGPGWGFGYLSAVLDDPAAAGTPQHAGTLQWGGVYGHSWFVDRALGLSVLLLTNTAYEGMSGPLTIALRDAVYAR')])),
             ('cif_file', Path('15k/folding/cifs/1ci8.cif')),
             ('a3m_files',
              [Path('15k/folding/MMSEQ_submission_second_try/1ci8_1.fa_results/uniref.a3m'),
               Path('15k/folding/MMSEQ_submission_second_try/1ci8_1.fa_resu